Домашнє завдання №2
==

В цьому завданні потрібно побудувати рекомендатор, використовуючи колаборативну фільтрацію. На вибір можна обрати наступні алгоритми із бібліотеки surprise
* KNNWithMeans - алгоритм, що враховує середні значення
* KNNWithZScore - алгоритм, що використовує нормалізацію
* SVD

Датасет - Book crossing. 

Фінальна функція приймає на вхід користувача (User-Id) та видає йому 10 рекомендацій книг. 

За бажанням можна також приділити більше уваги очищенню даних
* Прибрати дублікати
* Видалити схожі, але різні назви однієї і тієї ж книги

In [1]:
import pandas as pd
from plotly.graph_objects import *
import numpy as np
import sys 
from memory_profiler import memory_usage
import os, psutil


from surprise import Dataset
from surprise import Reader
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore, SVD
from surprise.model_selection import cross_validate, train_test_split, GridSearchCV

In [2]:
users = pd.read_csv('datasets/book-crossing/Users.csv', sep=';')
ratings = pd.read_csv('datasets/book-crossing/Ratings.csv',sep=';')#,on_bad_lines='skip')

books = pd.read_csv('datasets/book-crossing/Books.csv',sep=';',on_bad_lines='skip')


/tmp/ipykernel_1650582/350196873.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  users = pd.read_csv('datasets/book-crossing/Users.csv', sep=';')


In [3]:
users.isnull().sum()
ratings.isnull().sum()

User-ID    0
ISBN       0
Rating     0
dtype: int64

In [4]:
import re

# Define a function to extract age and country
def extract_age_country(data):
    age = re.findall(r'\b\d{1,3}\b', str(data))
    age = int(age[0]) if (age and 0 < int(age[0]) < 100) else 'nan'

    country = re.findall(r'(?i)\b[a-z ]+\b', str(data))
    country = country[0].strip() if country else 'nan'
    return age, country

# Apply the function to the column and split the results into separate columns
users[['Age', 'Country']] = users['Age'].apply(lambda x: pd.Series(extract_age_country(x)))

# Drop the original column and NaN rows
#users = users.drop(columns=['Age']).dropna()

In [5]:
users['Age'].unique()

array(['nan', 18, 17, 61, 26, 14, 25, 19, 46, 55, 32, 24, 20, 34, 23, 51,
       31, 21, 44, 30, 57, 43, 37, 41, 54, 42, 50, 39, 53, 47, 36, 28, 35,
       13, 58, 49, 38, 45, 62, 63, 27, 33, 29, 66, 40, 15, 60, 79, 22, 16,
       65, 59, 48, 72, 56, 67, 1, 80, 52, 69, 71, 73, 78, 9, 64, 12, 74,
       75, 3, 76, 83, 68, 11, 2, 70, 77, 93, 8, 7, 4, 81, 10, 5, 6, 96,
       84, 82, 90, 91, 94, 85, 97, 86, 92, 87, 95, 89, 99, 88, 98],
      dtype=object)

In [6]:
ratings = ratings.merge(users, on='User-ID')

In [7]:
ratings.head()

,User-ID,ISBN,Rating,Age,Country
0,2,0195153448,0,18,nan
1,7,034542252,0,nan,nan
2,8,0002005018,5,nan,nan
3,8,0060973129,0,nan,nan
4,8,0374157065,0,nan,nan


## KNNWithMeans

In [8]:
print("median amount users in groups--",ratings.groupby(['Rating', 'Age', 'Country'], dropna=False).size().reset_index()[0].median())
print("mean amount users in groups--",ratings.groupby(['Rating', 'Age', 'Country'], dropna=False).size().reset_index()[0].mean())

median amount users in groups-- 48.0
mean amount users in groups-- 986.5136612021857


In [9]:
Reader = Reader(line_format='user item rating',rating_scale=(1, 10))

In [10]:
ratings_non_null = ratings[ratings['Rating'] != 0]
ratings_non_null = ratings_non_null.iloc[:10000, [0, 1, 2]]
ratings_non_null.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 2 to 21398
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   User-ID  10000 non-null  object
 1   ISBN     10000 non-null  object
 2   Rating   10000 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 312.5+ KB


In [27]:
ratings_non_null.head()

,User-ID,ISBN,Rating
2,8,0002005018,5
10,8,074322678X,5
14,8,0887841740,5
15,8,1552041778,5
17,8,1567407781,6


In [11]:
def measure_memory(func):
    mem_before = psutil.Process(os.getpid()).memory_info().rss / 1024 / 1024
    result = func()
    mem_after = psutil.Process(os.getpid()).memory_info().rss / 1024 / 1024
    print(f"Memory used: {mem_after - mem_before} MB")
    return result

def build_trainset():
    return Dataset.load_from_df(ratings_non_null[['User-ID', 'ISBN', 'Rating']], Reader) 


data = measure_memory(build_trainset) 

#data = Dataset.load_from_df(ratings_non_null[['User-ID', 'ISBN', 'Rating']], Reader)

Memory used: 0.0 MB


In [35]:
sim_options = {'name': 'pearson_baseline', 'user_based': False}

algo = KNNWithZScore(k=50, sim_options=sim_options)

In [13]:
# algo = SVD()

In [37]:


def measure_memory(func):
    mem_before = psutil.Process(os.getpid()).memory_info().rss / 1024 / 1024
    result = func()
    mem_after = psutil.Process(os.getpid()).memory_info().rss / 1024 / 1024
    print(f"Memory used: {mem_after - mem_before} MB")
    return result

def build_trainset():
    return data.build_full_trainset()   

trainset = measure_memory(build_trainset)

algo.fit(trainset)   


# trainset = data.build_full_trainset()
# algo.fit(trainset)

Memory used: 0.0 MB
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [38]:
# Predict a certain item
uid = str(196)  # raw user id
iid = str("0002005018")  # raw item id
pred = algo.predict(uid, iid, r_ui=4, verbose=True)


user: 196        item: 0002005018 r_ui = 4.00   est = 7.63   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}


In [39]:
prediction = algo.predict(243,'0060915544')

In [40]:
prediction.est
pred.est

7.6295

In [41]:
prediction
pred

Prediction(uid='196', iid='0002005018', r_ui=4, est=7.6295, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})

In [42]:
def measure_memory(func):
    mem_before = psutil.Process(os.getpid()).memory_info().rss / 1024 / 1024
    result = func()
    mem_after = psutil.Process(os.getpid()).memory_info().rss / 1024 / 1024
    print(f"Memory used: {mem_after - mem_before} MB")
    return result

def build_trainset():
    return train_test_split(data, test_size=0.05)  #!

train, test = measure_memory(build_trainset)


#train, test = train_test_split(data, test_size=0.1)

Memory used: 0.0 MB


In [43]:
train

In [44]:
algo.fit(train)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [45]:
predictions = algo.test(test)

In [46]:
like_threshold = 7


hits = [1 for prediction in predictions if prediction.est >= like_threshold and prediction.r_ui >= like_threshold]
hit_rate = sum(hits) / len(predictions) if predictions else 0

print(f"Hit Rate: {hit_rate:.3f}")

Hit Rate: 0.746


In [47]:
def measure_memory(func):
    mem_before = psutil.Process(os.getpid()).memory_info().rss / 1024 / 1024
    result = func()
    mem_after = psutil.Process(os.getpid()).memory_info().rss / 1024 / 1024
    print(f"Memory used: {mem_after - mem_before} MB")
    return result

def build_trainset():
    return cross_validate(algo, data, measures=['RMSE'], cv=3, verbose=True)  #!

measure_memory(build_trainset)


#cross_validate(algo, data, measures=['RMSE'], cv=3, verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNWithZScore on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.9201  1.8465  1.9071  1.8912  0.0321  
Fit time          0.65    0.67    0.66    0.66    0.01    
Test time         0.02    0.02    0.01    0.02    0.00    
Memory used: -252.37890625 MB


{'test_rmse': array([1.92006428, 1.84648035, 1.90711215]),
 'fit_time': (0.6534712314605713, 0.6668829917907715, 0.6584157943725586),
 'test_time': (0.016679763793945312,
  0.016622304916381836,
  0.014945507049560547)}

In [53]:
sim_options = {
    "name": ["msd", "cosine"],
    "min_support": [3, 4, 5],
    "user_based": [False, True],
}

# sim_options = {
#     "name": ["msd"],
#     "min_support": [3],
#     "user_based": [False],
# }


param_grid = {"sim_options": sim_options}

gs = GridSearchCV(KNNWithMeans, param_grid, measures=["rmse", "mae"], cv=2, n_jobs=-1)

gs.fit(data)

Computing the msd similarity matrix...
Computing the msd similarity matrix...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Computing the msd similarity matrix...
Computing the msd similarity matrix...
Computing the msd similarity matrix...
Computing the msd similarity matrix...
Done computing similarity matrix.
Done computing similarity matrix.
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Computing the msd similarity matrix...
Computing the cosine similarity matrix...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Computing the cosine similarity matrix...
Computing the cosine similarity matrix...
Computing the cosine similarity matrix...
Computing the cosine similarity matrix...


In [54]:
print(gs.best_score["rmse"])
print(gs.best_params["rmse"])

1.8173268057957617
{'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': True}}


In [55]:
# Get the best parameters
best_params = gs.best_params['rmse']

# Create a new model with the best parameters
best_model = KNNWithMeans(sim_options=best_params['sim_options'])

# Fit the model to the data
best_model.fit(data.build_full_trainset())

# Get the id of the item you're interested in
item_id = '0002005018'

# Get the inner id of the item
inner_id = best_model.trainset.to_inner_iid(item_id)

# Get the top 10 items similar to the given item
neighbors = best_model.get_neighbors(inner_id, 10)

# Convert the inner ids of the neighbors into raw ids
neighbors = [best_model.trainset.to_raw_iid(inner_id) for inner_id in neighbors]

print(neighbors)

Computing the msd similarity matrix...
Done computing similarity matrix.
['074322678X', '0887841740', '1552041778', '1567407781', '1575663937', '1881320189', '0452264464', '8477024456', '1879384493', '0061076031']


In [58]:
# Get the best parameters
best_params = gs.best_params['rmse']

# Create a new model with the best parameters
best_model = KNNWithMeans(sim_options=best_params['sim_options'])

# Fit the model to the data
best_model.fit(data.build_full_trainset())

# Get the id of the user you're interested in
user_id = 8

# Get the inner id of the user
inner_id = best_model.trainset.to_inner_uid(user_id)

# Get the top 10 users similar to the given user
neighbors = best_model.get_neighbors(inner_id, 10)

# Convert the inner ids of the neighbors into raw ids
neighbors = [best_model.trainset.to_raw_uid(inner_id) for inner_id in neighbors]

print(neighbors)

Computing the msd similarity matrix...
Done computing similarity matrix.
[9, 10, 12, 14, 16, 17, 19, 22, 26, 32]


In [59]:
# Get the best parameters
best_params = gs.best_params['rmse']

# Create a new model with the best parameters
best_model = KNNWithMeans(sim_options=best_params['sim_options'])

# Fit the model to the data
best_model.fit(data.build_full_trainset())

# Get the id of the user you're interested in
user_id = 8

# Get the list of all item ids
item_ids = best_model.trainset.all_items()

# Predict the rating for each item
predictions = [(iid, best_model.predict(user_id, best_model.trainset.to_raw_iid(iid)).est) for iid in item_ids]

# Sort the predictions by estimated rating in descending order
predictions.sort(key=lambda x: x[1], reverse=True)

# Get the top 10 items
top_items = predictions[:10]

print(top_items)

Computing the msd similarity matrix...
Done computing similarity matrix.
[(6, 7.0), (4, 6.0), (5, 6.0), (7, 5.571428571428571), (8, 5.571428571428571), (9, 5.571428571428571), (10, 5.571428571428571), (11, 5.571428571428571), (12, 5.571428571428571), (13, 5.571428571428571)]


In [60]:
# Get the best parameters
best_params = gs.best_params['rmse']

# Create a new model with the best parameters
best_model = KNNWithMeans(sim_options=best_params['sim_options'])

# Fit the model to the data
best_model.fit(data.build_full_trainset())

# Get the id of the user you're interested in
user_id = 8

# Get the list of all item ids
item_ids = best_model.trainset.all_items()

# Predict the rating for each item
predictions = [(best_model.trainset.to_raw_iid(iid), best_model.predict(user_id, best_model.trainset.to_raw_iid(iid)).est) for iid in item_ids]

# Sort the predictions by estimated rating in descending order
predictions.sort(key=lambda x: x[1], reverse=True)

# Get the top 10 items
top_items = predictions[:10]

# Get the ids of the top items
top_item_ids = [item_id for (item_id, _) in top_items]



Computing the msd similarity matrix...
Done computing similarity matrix.


In [61]:
top_item_ids

['1881320189',
 '1567407781',
 '1575663937',
 '0452264464',
 '8477024456',
 '1879384493',
 '0061076031',
 '0439095026',
 '0689821166',
 '0345402871']

In [63]:
# Filter the dataframe to include only the top items
recommended_items = ratings_non_null[ratings_non_null['ISBN'].isin(top_item_ids)]

recommended_items

,User-ID,ISBN,Rating
17,8,1567407781,6
18,8,1575663937,6
19,8,1881320189,7
21,9,0452264464,6
24,10,8477024456,6
25,12,1879384493,10
26,14,0061076031,5
27,14,0439095026,5
28,14,0689821166,6
30,16,0345402871,9


In [66]:
books[books['ISBN'].isin(top_item_ids)].merge(recommended_items, on='ISBN')

,ISBN,Title,Author,Year,Publisher,User-ID,Rating
0,1567407781,The Witchfinder (Amos Walker Mystery Series),Loren D. Estleman,1998,Brilliance Audio - Trade,8,6
1,1575663937,More Cunning Than Man: A Social History of Rat...,Robert Hendrickson,1999,Kensington Publishing Corp.,8,6
2,1881320189,Goodbye to the Buttermilk Sky,Julia Oliver,1994,River City Pub,8,7
3,0452264464,Beloved (Plume Contemporary Fiction),Toni Morrison,1994,Plume,9,6
4,0452264464,Beloved (Plume Contemporary Fiction),Toni Morrison,1994,Plume,3373,10
5,0452264464,Beloved (Plume Contemporary Fiction),Toni Morrison,1994,Plume,5555,9
6,0452264464,Beloved (Plume Contemporary Fiction),Toni Morrison,1994,Plume,6242,5
7,1879384493,If I'd Known Then What I Know Now: Why Not Lea...,J. R. Parrish,2003,Cypress House,12,10
8,0061076031,Mary-Kate & Ashley Switching Goals (Mary-Kate ...,Mary-Kate &amp; Ashley Olsen,2000,HarperEntertainment,14,5
9,0439095026,Tell Me This Isn't Happening,Robynn Clairday,1999,Scholastic,14,5
